<a href="https://colab.research.google.com/github/dallalysalah/TP-Maths/blob/main/tp3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
from scipy.optimize import linprog


def generate_tabinitial(A, b, c):

    A = np.array(A, dtype=float)
    b = np.array(b, dtype=float).reshape(-1, 1)
    c = np.array(c, dtype=float)

    m, n = A.shape


    I = np.eye(m)
    A_bar = np.hstack([A, I])



    c_bar = np.concatenate([c, np.zeros(m)])


    T = np.zeros((m + 1, n + m + 1))

    # Contraintes
    T[:m, :n + m] = A_bar
    T[:m, -1] = b.flatten()

    # Ligne des coûts réduits initiale
    T[-1, :n + m] = c_bar
    T[-1, -1] = 0.0

    return T


def positivite(v):
    """
    v : vecteur (1D ou 2D)
    Retourne :
      flag_pos : True si toutes les composantes >= 0
      min_val  : valeur minimale de v
      idx_min  : indice de la valeur minimale
    """
    v = np.array(v, dtype=float).flatten()
    min_val = np.min(v)
    idx_min = np.argmin(v)
    flag_pos = np.all(v >= 0)
    return flag_pos, min_val, idx_min


def rapportmin(b, a):
    """
    b, a : vecteurs colonnes (ou 1D) de même taille.
    Retourne l'indice i du rapport minimum positif b_i / a_i
    avec a_i > 0.

    Si aucun a_i > 0 -> retourne None (problème non borné).
    """
    b = np.array(b, dtype=float).flatten()
    a = np.array(a, dtype=float).flatten()

    ratios = []
    indices = []

    for i in range(len(b)):
        if a[i] > 0:
            ratios.append(b[i] / a[i])
            indices.append(i)

    if len(ratios) == 0:
        return None  # pas de rapport positif -> non borné

    ratios = np.array(ratios)
    idx_local = np.argmin(ratios)
    return indices[idx_local]


def pivotgauss(T, r, s):
    """
    Effectue un pivot de Gauss sur le tableau T
    en utilisant le pivot situé en (r, s).
    r, s : indices de ligne et colonne (en base 0).

    Modifie T sur place et le retourne aussi.
    """
    T = np.array(T, dtype=float)
    m, n = T.shape

    pivot = T[r, s]
    if pivot == 0:
        raise ValueError("Pivot nul, pivot de Gauss impossible.")

    # 1) Normaliser la ligne pivot
    T[r, :] = T[r, :] / pivot

    # 2) Annuler les autres éléments de la colonne pivot
    for i in range(m):
        if i != r:
            facteur = T[i, s]
            T[i, :] = T[i, :] - facteur * T[r, :]

    return T


def simplexe(A, b, c, max_iter=100):
    """
    Implémente l’algorithme du simplexe primal pour un problème de MIN
      min c^T x
      s.c. A x <= b
           x >= 0

    A, b, c : comme dans generate_tabinitial.

    Retourne :
      x_opt : solution optimale (variables de décision uniquement)
      z_opt : valeur optimale de la fonction objectif
      T     : tableau final du simplexe
    """
    A = np.array(A, dtype=float)
    b = np.array(b, dtype=float)
    c = np.array(c, dtype=float)

    m, n = A.shape

    # Tableau initial
    T = generate_tabinitial(A, b, c)

    # Base initiale : les variables d'écart (indices n..n+m-1)
    base = list(range(n, n + m))

    for it in range(max_iter):
        # Coefficients réduits (dernière ligne sans la colonne b)
        cbar = T[-1, :-1]

        # Choix de la colonne pivot (variable qui entre en base)
        all_pos, min_val, s = positivite(cbar)

        # Si tous les coefficients réduits >= 0 : optimalité atteinte
        if all_pos:
            z_opt = T[-1, -1]
            # Reconstruire la solution x (n variables de décision)
            x_opt = np.zeros(n)
            bbar = T[:-1, -1]
            for i, var_index in enumerate(base):
                if var_index < n:  # variable de décision (pas une slack)
                    x_opt[var_index] = bbar[i]
            return x_opt, z_opt, T

        # Sinon, la colonne pivot est s (où cbar est minimal, donc négatif)
        # Choix de la ligne pivot (variable qui sort de la base)
        col_s = T[:-1, s]      # colonne s sans la dernière ligne
        bbar = T[:-1, -1]      # colonne b sans la dernière ligne

        r = rapportmin(bbar, col_s)
        if r is None:
            raise ValueError("Problème non borné inférieurement (aucun rapport positif).")

        # Pivot de Gauss sur (r, s)
        T = pivotgauss(T, r, s)

        # Mise à jour de la base
        base[r] = s

    raise RuntimeError("Nombre maximal d'itérations atteint dans le simplexe.")


# ============================================================
# 4.5 Exemple d’utilisation
# ============================================================

# Problème :
# min z = -3 x1 - 2 x2
# s.c.   x1 +  x2 <= 4
#        x1 + 2x2 <= 6
#        x1, x2 >= 0

A_ex = np.array([[1, 1],
                 [1, 2]], dtype=float)
b_ex = np.array([4, 6], dtype=float)
c_ex = np.array([-3, -2], dtype=float)  # on minimise -3x1 -2x2

x_opt, z_opt, T_final = simplexe(A_ex, b_ex, c_ex)

print("=== Résultat avec simplexe maison ===")
print("x* =", x_opt)
print("z* =", z_opt)
print("Tableau final :")
print(T_final)


res = linprog(c_ex, A_ub=A_ex, b_ub=b_ex)
print("\n=== Résultat avec scipy.optimize.linprog ===")
print(res)
print("Optimal value :", res.fun, "\nX :", res.x)


=== Résultat avec simplexe maison ===
x* = [4. 0.]
z* = 12.0
Tableau final :
[[ 1.  1.  1.  0.  4.]
 [ 0.  1. -1.  1.  2.]
 [ 0.  1.  3.  0. 12.]]

=== Résultat avec scipy.optimize.linprog ===
        message: Optimization terminated successfully. (HiGHS Status 7: Optimal)
        success: True
         status: 0
            fun: -12.0
              x: [ 4.000e+00  0.000e+00]
            nit: 2
          lower:  residual: [ 4.000e+00  0.000e+00]
                 marginals: [ 0.000e+00  1.000e+00]
          upper:  residual: [       inf        inf]
                 marginals: [ 0.000e+00  0.000e+00]
          eqlin:  residual: []
                 marginals: []
        ineqlin:  residual: [ 0.000e+00  2.000e+00]
                 marginals: [-3.000e+00 -0.000e+00]
 mip_node_count: 0
 mip_dual_bound: 0.0
        mip_gap: 0.0
Optimal value : -12.0 
X : [4. 0.]


In [ ]:
import numpy as np
from scipy.optimize import linprog


def generate_tabinitial(A, b, c):

    A = np.array(A, dtype=float)
    b = np.array(b, dtype=float).reshape(-1, 1)
    c = np.array(c, dtype=float)

    m, n = A.shape


    I = np.eye(m)
    A_bar = np.hstack([A, I])




    c_bar = np.concatenate([c, np.zeros(m)])


    T = np.zeros((m + 1, n + m + 1))

    # Contraintes
    T[:m, :n + m] = A_bar
    T[:m, -1] = b.flatten()





    # Ligne des coûts réduits initiale
    T[-1, :n + m] = c_bar
    T[-1, -1] = 0.0

    return T


def positivite(v):
    """
    v : vecteur (1D ou 2D)
    Retourne :
      flag_pos : True si toutes les composantes >= 0
      min_val  : valeur minimale de v
      idx_min  : indice de la valeur minimale
    """
    v = np.array(v, dtype=float).flatten()
    min_val = np.min(v)
    idx_min = np.argmin(v)
    flag_pos = np.all(v >= 0)
    return flag_pos, min_val, idx_min


def rapportmin(b, a):
    """
    b, a : vecteurs colonnes (ou 1D) de même taille.
    Retourne l'indice i du rapport minimum positif b_i / a_i
    avec a_i > 0.

    Si aucun a_i > 0 -> retourne None (problème non borné).
    """
    b = np.array(b, dtype=float).flatten()
    a = np.array(a, dtype=float).flatten()

    ratios = []
    indices = []

    for i in range(len(b)):
        if a[i] > 0:
            ratios.append(b[i] / a[i])
            indices.append(i)

    if len(ratios) == 0:
        return None  # pas de rapport positif -> non borné

    ratios = np.array(ratios)
    idx_local = np.argmin(ratios)
    return indices[idx_local]


def pivotgauss(T, r, s):
    """
    Effectue un pivot de Gauss sur le tableau T
    en utilisant le pivot situé en (r, s).
    r, s : indices de ligne et colonne (en base 0).

    Modifie T sur place et le retourne aussi.
    """
    T = np.array(T, dtype=float)
    m, n = T.shape

    pivot = T[r, s]
    if pivot == 0:
        raise ValueError("Pivot nul, pivot de Gauss impossible.")

    # 1) Normaliser la ligne pivot
    T[r, :] = T[r, :] / pivot

    # 2) Annuler les autres éléments de la colonne pivot
    for i in range(m):
        if i != r:
            facteur = T[i, s]
            T[i, :] = T[i, :] - facteur * T[r, :]

    return T


def simplexe(A, b, c, max_iter=100):
    """
    Implémente l’algorithme du simplexe primal pour un problème de MIN
      min c^T x
      s.c. A x <= b
           x >= 0

    A, b, c : comme dans generate_tabinitial.

    Retourne :
      x_opt : solution optimale (variables de décision uniquement)
      z_opt : valeur optimale de la fonction objectif
      T     : tableau final du simplexe
    """
    A = np.array(A, dtype=float)
    b = np.array(b, dtype=float)
    c = np.array(c, dtype=float)

    m, n = A.shape

    # Tableau initial
    T = generate_tabinitial(A, b, c)

    # Base initiale : les variables d'écart (indices n..n+m-1)
    base = list(range(n, n + m))

    for it in range(max_iter):
        # Coefficients réduits (dernière ligne sans la colonne b)
        cbar = T[-1, :-1]

        # Choix de la colonne pivot (variable qui entre en base)
        all_pos, min_val, s = positivite(cbar)

        # Si tous les coefficients réduits >= 0 : optimalité atteinte
        if all_pos:
            z_opt = T[-1, -1]
            # Reconstruire la solution x (n variables de décision)
            x_opt = np.zeros(n)
            bbar = T[:-1, -1]
            for i, var_index in enumerate(base):
                if var_index < n:  # variable de décision (pas une slack)
                    x_opt[var_index] = bbar[i]
            return x_opt, z_opt, T

        # Sinon, la colonne pivot est s (où cbar est minimal, donc négatif)
        # Choix de la ligne pivot (variable qui sort de la base)
        col_s = T[:-1, s]      # colonne s sans la dernière ligne
        bbar = T[:-1, -1]      # colonne b sans la dernière ligne

        r = rapportmin(bbar, col_s)
        if r is None:
            raise ValueError("Problème non borné inférieurement (aucun rapport positif).")

        # Pivot de Gauss sur (r, s)
        T = pivotgauss(T, r, s)

        # Mise à jour de la base
        base[r] = s

    raise RuntimeError("Nombre maximal d'itérations atteint dans le simplexe.")


# ============================================================
# 4.5 Exemple d’utilisation
# ============================================================

# Problème :
# min z = -3 x1 - 2 x2
# s.c.   x1 +  x2 <= 4
#        x1 + 2x2 <= 6
#        x1, x2 >= 0

A_ex = np.array([[1, 1],
                 [1, 2]], dtype=float)
b_ex = np.array([4, 6], dtype=float)
c_ex = np.array([-3, -2], dtype=float)  # on minimise -3x1 -2x2

x_opt, z_opt, T_final = simplexe(A_ex, b_ex, c_ex)

print("=== Résultat avec simplexe maison ===")
print("x* =", x_opt)
print("z* =", z_opt)
print("Tableau final :")
print(T_final)


res = linprog(c_ex, A_ub=A_ex, b_ub=b_ex)
print("\n=== Résultat avec scipy.optimize.linprog ===")
print(res)
print("Optimal value :", res.fun, "\nX :", res.x)


=== Résultat avec simplexe maison ===
x* = [4. 0.]
z* = 12.0
Tableau final :
[[ 1.  1.  1.  0.  4.]
 [ 0.  1. -1.  1.  2.]
 [ 0.  1.  3.  0. 12.]]

=== Résultat avec scipy.optimize.linprog ===
        message: Optimization terminated successfully. (HiGHS Status 7: Optimal)
        success: True
         status: 0
            fun: -12.0
              x: [ 4.000e+00  0.000e+00]
            nit: 2
          lower:  residual: [ 4.000e+00  0.000e+00]
                 marginals: [ 0.000e+00  1.000e+00]
          upper:  residual: [       inf        inf]
                 marginals: [ 0.000e+00  0.000e+00]
          eqlin:  residual: []
                 marginals: []
        ineqlin:  residual: [ 0.000e+00  2.000e+00]
                 marginals: [-3.000e+00 -0.000e+00]
 mip_node_count: 0
 mip_dual_bound: 0.0
        mip_gap: 0.0
Optimal value : -12.0 
X : [4. 0.]
